In [12]:
import cv2
import imageio as ig
import numpy as np
import time
import random
import string
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu

In [13]:
# global variables

xLim = 25
yLim = 25

# bounds for cscaling the data
lower = 1
upper = 2

# dimensions fo input image for network
dimension = 28

# minimun to use for the scaling of the data
minn = 0
# line width
l = 1

norm = 255.0

# anser
ans = list(string.ascii_uppercase)

# Here we are defining range of bluecolor in HSV 
lower_blue = np.array([26,15,72]) 
upper_blue = np.array([179,255,255]) 

# path to model
save_path = "./ModelSaver"

In [57]:
# function that takes in image and color to mask and dimen of output (asssumed square) 
# and outputs network ready image,
def getReady(frame, lower, upper, dimen):
    thresh = 150
    kernel = np.ones((3,3),np.uint8)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 
    mask = cv2.inRange(hsv, lower, upper) 

    # The bitwise and of the frame and mask is done so  
    # that only the blue coloured objects are highlighted  
    # and stored in res 
    res = cv2.bitwise_and(frame,frame, mask= mask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY) 
    inv = cv2.bitwise_not(mask)

    im_bw = cv2.threshold((gray+inv) , thresh, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]

    im_bw = cv2.erode(im_bw,kernel,iterations = 1)

    kernel = np.ones((2,2),np.uint8)
    im_bw = cv2.dilate(im_bw,kernel,iterations = 1)

    im_bw = cv2.resize(im_bw, (dimen,dimen), interpolation=cv2.INTER_NEAREST)
#     im_bw = cv2.threshold((im_bw) , thresh, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
#     im_bw = cv2.morphologyEx(im_bw, cv2.MORPH_OPEN, kernel)
#     im_bw = cv2.morphologyEx(im_bw, cv2.MORPH_CLOSE, kernel)

    return im_bw


In [39]:
# prediction

def getPrediction(session, image, answer, dimen):
    print("getting prediction")
    theImg = image.reshape(-1, dimen, dimen, 1)
#     tf.reset_default_graph()
    # loading model
#     with tf.Session(graph=tf.Graph()) as sess:
#         tf.saved_model.loader.load(sess,  ["tag"], model)
#         graph = tf.get_default_graph()

    x = graph.get_tensor_by_name("x:0")
    myOut = graph.get_tensor_by_name("myOutput:0")


    yOut = session.run(myOut, feed_dict={x:theImg})
    

    ind = np.where(yOut[0] == np.amax(yOut[0]))
    print(np.array(answer)[np.array(ind[0])])
    return answer[int(ind[0][0])]

In [40]:
# loading model
tf.reset_default_graph()
graph = tf.get_default_graph()
sess = tf.Session()
tf.saved_model.loader.load(sess,  ["tag"], save_path)
graph = tf.get_default_graph()
print("Model Loaded")


INFO:tensorflow:Restoring parameters from ./ModelSaver/variables/variables
Model Loaded


In [61]:
img = cv2.imread('/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/Images/image10.JPG',1)
final = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [62]:
width, height = final.shape
ret,thresh = cv2.threshold(final,127,255,0)
contours,hierarchy = cv2.findContours(thresh, 1, 2)

In [63]:
# loop over the contours
for c in contours:
    
    x,y,w,h = cv2.boundingRect(c)
    
    # converting and then scaling between lower an upper
    trX = ((upper - lower)/(width - minn))*(width-x) + lower
    trY = ((upper - lower)/(height - minn))*(height-y) + lower
    
    # getting random color
    color = (random.randint(0,255), random.randint(10,255), random.randint(0,255))
    
    # see if big enough for it to be alphabet
    if ((w >= xLim*trX)&(h >= yLim*trY)):
        
        toNet = getReady(img[y:y+h, x:x+w], lower_blue, upper_blue, dimension)
        
#         cv2.imshow("Input", toNet)
        
        pred = getPrediction(sess, (toNet/norm), ans, dimension)
        
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(0, 0, 255),l) 
        img = cv2.putText(img, pred, (int(x+w),int(y)), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1)
        
#         cv2.imshow("Image", img)      

#         key = cv2.waitKey(0)
#         if key == 27:#if ESC is pressed, exit loop
#             cv2.destroyAllWindows()
#             break
#         elif key == 82:
#             cv2.destroyAllWindows()
            
#         break
cv2.imshow("Image", img)  
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("realTest.jpg", img)


getting prediction
['Z']
getting prediction
['X']
getting prediction
['Y']
getting prediction
['Z']
getting prediction
['T']
getting prediction
['U']
getting prediction
['H']
getting prediction
['V']
getting prediction
['W']
getting prediction
['O']
getting prediction
['W']
getting prediction
['M']
getting prediction
['V']
getting prediction
['U']
getting prediction
['I']
getting prediction
['K']
getting prediction
['J']
getting prediction
['L']
getting prediction
['Z']
getting prediction
['G']
getting prediction
['S']
getting prediction
['F']
getting prediction
['C']
getting prediction
['C']
getting prediction
['S']
getting prediction
['R']


True

In [125]:
sess.close()